In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.functions import *

# warehouse_location points to the default location for managed databases and tables
warehouse_location = 'hdfs://hdfs-nn:9000/warehouse'

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL Hive integration example") \
    .config("spark.sql.warehouse.dir", warehouse_location) \
    .config("hive.metastore.uris", "thrift://hive-metastore:9083") \
    .enableHiveSupport() \
    .getOrCreate()

In [2]:
spark.sql(
    """
    SHOW TABLES FROM tabd_db
    """
).show()

+--------+--------------------+-----------+
|database|           tableName|isTemporary|
+--------+--------------------+-----------+
| tabd_db|distributiongloba...|      false|
| tabd_db|globalmobilepenet...|      false|
| tabd_db|globalsocialnetworks|      false|
| tabd_db| internetactivities1|      false|
| tabd_db| internetactivities2|      false|
| tabd_db|       internetusage|      false|
| tabd_db|numberofworldwide...|      false|
| tabd_db|numberofworldwide...|      false|
| tabd_db|p_dailyinternetus...|      false|
| tabd_db|p_enterprisesocia...|      false|
| tabd_db|p_globalsocialmed...|      false|
| tabd_db|p_globalsocialnet...|      false|
| tabd_db|p_individualsacti...|      false|
| tabd_db|     p_internetusage|      false|
| tabd_db|parentawarenessof...|      false|
| tabd_db|socialmediausagef...|      false|
| tabd_db|socialmediausebyp...|      false|
| tabd_db|socialmediausebyt...|      false|
| tabd_db|socialmediausersa...|      false|
| tabd_db|  socialnetworkusers| 

In [3]:
spark.sql(
    """
    DROP TABLE IF EXISTS tabd_db.InternetUsageFrequency
    """
)

spark.sql(
    """
    CREATE EXTERNAL TABLE tabd_db.InternetUsageFrequency (
        year INT,
        region VARCHAR(45),
        gender VARCHAR(45),
        usage_rate DOUBLE,
        penetration_percentage INT,
        internet_users DOUBLE,
        frequency_of_access VARCHAR(45),
        individuals VARCHAR(45),
        frequency_of_access_percentage INT

    )
    STORED AS PARQUET
    LOCATION 'hdfs://hdfs-nn:9000/warehouse/tabd.db/InternetUsageFrequency/'
    """
)

        

# tblproperties('skip.header.line.count'='1')
# can be used for csvs with header
# but spark sql cannot understand that at the moment, while reading the data using sql queries
# so just when creating hive tables backed up by csvs, avoid headers
# in the project we wont use hive text tables, so all good.

DataFrame[]

In [4]:
spark.sql(
    """
    SHOW TABLES FROM tabd_db
    """
).show()

+--------+--------------------+-----------+
|database|           tableName|isTemporary|
+--------+--------------------+-----------+
| tabd_db|distributiongloba...|      false|
| tabd_db|globalmobilepenet...|      false|
| tabd_db|globalsocialnetworks|      false|
| tabd_db| internetactivities1|      false|
| tabd_db| internetactivities2|      false|
| tabd_db|       internetusage|      false|
| tabd_db|internetusagefreq...|      false|
| tabd_db|numberofworldwide...|      false|
| tabd_db|numberofworldwide...|      false|
| tabd_db|p_dailyinternetus...|      false|
| tabd_db|p_enterprisesocia...|      false|
| tabd_db|p_globalsocialmed...|      false|
| tabd_db|p_globalsocialnet...|      false|
| tabd_db|p_individualsacti...|      false|
| tabd_db|     p_internetusage|      false|
| tabd_db|parentawarenessof...|      false|
| tabd_db|socialmediausagef...|      false|
| tabd_db|socialmediausebyp...|      false|
| tabd_db|socialmediausebyt...|      false|
| tabd_db|socialmediausersa...| 

In [5]:
# Let's look into HDFS

In [6]:
spark.sql(
    """
    SELECT *
    FROM tabd_db.InternetUsageFrequency
    """
).show()

+----+--------------------+------+----------+----------------------+--------------+--------------------+--------------------+------------------------------+
|year|              region|gender|usage_rate|penetration_percentage|internet_users| frequency_of_access|         individuals|frequency_of_access_percentage|
+----+--------------------+------+----------+----------------------+--------------+--------------------+--------------------+------------------------------+
|2011|European Union fr...|  null|       0.0|                     0|           0.0|         Once a week|     All Individuals|                            65|
|2011|European Union fr...|  null|       0.0|                     0|           0.0|         Once a week|All individuals w...|                            94|
|2011|European Union fr...|  null|       0.0|                     0|           0.0|               Daily|     All Individuals|                            54|
|2011|European Union fr...|  null|       0.0|             

In [7]:
spark.sql(
    """
    DESCRIBE FORMATTED tabd_db.InternetUsageFrequency
    """
).toPandas()

col_name  \
0                             year   
1                           region   
2                           gender   
3                       usage_rate   
4           penetration_percentage   
5                   internet_users   
6              frequency_of_access   
7                      individuals   
8   frequency_of_access_percentage   
9                                    
10    # Detailed Table Information   
11                        Database   
12                           Table   
13                           Owner   
14                    Created Time   
15                     Last Access   
16                      Created By   
17                            Type   
18                        Provider   
19                Table Properties   
20                      Statistics   
21                        Location   
22                   Serde Library   
23                     InputFormat   
24                    OutputFormat   
25              Storage Properties   
26              Partition Provider   

                                            data_type comment  
0                                                 int    None  
1                                         varchar(45)    None  
2                                         varchar(45)    None  
3                                              double    None  
4                                                 int    None  
5                                              double    None  
6                                         varchar(45)    None  
7                                         varchar(45)    None  
8                                                 int    None  
9                                                              
10                                                             
11                                            tabd_db          
12                             internetusagefrequency          
13                                             jovyan          
14                       Wed Jun 09 13:27:01 UTC 2021          
15                                            UNKNOWN          
16                                        Spark 3.1.1          
17                                           EXTERNAL          
18                                               hive          
19                 [transient_lastDdlTime=1623245221]          
20                                         8255 bytes          
21  hdfs://hdfs-nn:9000/warehouse/tabd.db/Internet...          
22  org.apache.hadoop.hive.ql.io.parquet.serde.Par...          
23  org.apache.hadoop.hive.ql.io.parquet.MapredPar...          
24  org.apache.hadoop.hive.ql.io.parquet.MapredPar...          
25                           [serialization.format=1]          
26                                            Catalog

In [8]:
# Let's put the files into HDFS

In [9]:
spark.sql(
    """
    SELECT *
    FROM tabd_db.InternetUsageFrequency
    """
).toPandas()

year                    region gender  usage_rate  \
0     2011  European Union from 2020   null         0.0   
1     2011  European Union from 2020   null         0.0   
2     2011  European Union from 2020   null         0.0   
3     2011  European Union from 2020   null         0.0   
4     2011  European Union from 2020   null         0.0   
...    ...                       ...    ...         ...   
5275  2020                    Kosovo   null         0.0   
5276  2020                    Kosovo   null         0.0   
5277  2020                    Kosovo   null         0.0   
5278  2020                    Kosovo   null         0.0   
5279  2020                    Kosovo   null         0.0   

      penetration_percentage  internet_users     frequency_of_access  \
0                          0             0.0             Once a week   
1                          0             0.0             Once a week   
2                          0             0.0                   Daily   
3                          0             0.0                   Daily   
4                          0             0.0    At least once a week   
...                      ...             ...                     ...   
5275                       0             0.0   At least once a month   
5276                       0             0.0  Less than once a month   
5277                       0             0.0  Less than once a month   
5278                       0             0.0   Less than once a week   
5279                       0             0.0   Less than once a week   

                                            individuals  \
0                                       All Individuals   
1     All individuals who used internet in the last ...   
2                                       All Individuals   
3     All individuals who used internet in the last ...   
4                                       All Individuals   
...                                                 ...   
5275  All individuals who used internet in the last ...   
5276                                    All Individuals   
5277  All individuals who used internet in the last ...   
5278                                    All Individuals   
5279  All individuals who used internet in the last ...   

      frequency_of_access_percentage  
0                                 65  
1                                 94  
2                                 54  
3                                 78  
4                                 11  
...                              ...  
5275                               0  
5276                               0  
5277                               0  
5278                               1  
5279                               1  

[5280 rows x 9 columns]

In [10]:


spark.sql(
    """
    SELECT *
    FROM tabd_db.InternetUsageFrequency
    """
).show()

+----+--------------------+------+----------+----------------------+--------------+--------------------+--------------------+------------------------------+
|year|              region|gender|usage_rate|penetration_percentage|internet_users| frequency_of_access|         individuals|frequency_of_access_percentage|
+----+--------------------+------+----------+----------------------+--------------+--------------------+--------------------+------------------------------+
|2011|European Union fr...|  null|       0.0|                     0|           0.0|         Once a week|     All Individuals|                            65|
|2011|European Union fr...|  null|       0.0|                     0|           0.0|         Once a week|All individuals w...|                            94|
|2011|European Union fr...|  null|       0.0|                     0|           0.0|               Daily|     All Individuals|                            54|
|2011|European Union fr...|  null|       0.0|             

In [11]:
spark.stop()